# Example usage

## Initialize a GeslaDataset object
Place the `gesla.py` file in your working directory (or elsewhere on your path), and import the `GeslaDataset` class. Selecting and loading data files requires paths to the metadata .csv file and the directory containing the data files. Initialize a `GeslaDataset` object with these paths as follows.

In [1]:
from gesla import GeslaDataset

meta_file = "../GESLAv3/GESLA3_ALL.csv"
data_path = "../GESLAv3/data/"

g3 = GeslaDataset(meta_file=meta_file, data_path=data_path)

## Load data from a single file
If you want to work with data from a single record, and you know the filename you want, use the function `file_to_pandas` as follows. The function returns a `pandas.DataFrame` with data and flags, and a `pandas.Series` containing metadata.

In [2]:
filename = "abrams_river-380-canada-meds"
data, meta = g3.file_to_pandas(filename)
data

,sea_level,qc_flag,use_flag
date_time,,,
1971-06-09 05:00:00,2.29,1,1
1971-06-09 06:00:00,1.79,1,1
1971-06-09 07:00:00,1.29,1,1
1971-06-09 08:00:00,0.79,1,1
1971-06-09 09:00:00,0.63,1,1
...,...,...,...
1971-10-26 00:00:00,0.86,1,1
1971-10-26 01:00:00,0.65,1,1
1971-10-26 02:00:00,0.99,1,1


## Load data from a list of files
If you want to work with data from multiple files, and you know the filenames you want, use the function `files_to_xarray` as follows. The function returns a `xarray.Dataset` object containing data, flags, and metadata.

In [3]:
filenames = [
    "abrams_river-380-canada-meds",
    "acajutla-082c-el_salvador-uhslc", 
    "yoshioka-hd26-japan-jodc_jcg", 
    "west_point_a_la_hache-8761494-united_states_of_america_the-noaa",
]
xr_dataset = g3.files_to_xarray(filenames)
print(xr_dataset)

<xarray.Dataset>
Dimensions:                  (date_time: 289251, station: 4)
Coordinates:
  * station                  (station) int64 0 1 2 3
  * date_time                (date_time) datetime64[ns] 1971-06-09T05:00:00 ....
Data variables:
    sea_level                (station, date_time) float64 2.29 1.79 ... nan nan
    qc_flag                  (station, date_time) float64 1.0 1.0 ... nan nan
    use_flag                 (station, date_time) float64 1.0 1.0 ... nan nan
    site_name                (station) object 'Acajutla' ... 'Abrams_River'
    site_code                (station) object '082C' 'HD26' '8761494' '380'
    country                  (station) object 'El_Salvador' 'Japan' ... 'Canada'
    contributor_abbreviated  (station) object 'UHSLC' 'JODC_JCG' 'NOAA' 'MEDS'
    contributor_full         (station) object 'University of Hawaii Sea Level...
    contributor_website      (station) object 'https://uhslc.soest.hawaii.edu...
    contributor_contact      (station) object 'ph

## Load data from the N closest records to a lat/lon location
Load data from records close to a particular location using the function `load_N_closest` as follows. Provide a lat/lon location and the number of desired records. The function returns a `xarray.Dataset` object containing data, flags, and metadata.  

Note the `UserWarning` that occurs when duplicate timestamps are encountered. The function `file_to_pandas` used to read each individual file keeps only the first of any duplicate timestamps.

In [4]:
data = g3.load_N_closest(lat=43.83, lon=-65.95, N=10)
print(data.site_name.values)

['Yarmouth' 'Lower_Wedgeport' 'Pinkney_Point' 'Abbotts_Harbour'
 'Clarks_Harbour' 'Woods_Harbour' 'Tusket' 'Abrams_River' 'Flat_Island'
 'Wedgeport']


/Users/phil/Dropbox (Personal)/Research/Data/TideGaugeSets/GESLAv3/GeslaDataset/gesla.py:76: UserWarning: Duplicate timestamps in file yarmouth-365-canada-meds were removed.
  warnings.warn(


## Load data from the records in a lat/lon range
Load data from records in a rectangular lat/lon range using the function `load_lat_lon_range` as follows. Provide lat/lon extents of the range. The function returns a `xarray.Dataset` object containing data, flags, and metadata.

In [5]:
south_lat = 15
north_lat = 30
west_lon = -180
east_lon = -140

data = g3.load_lat_lon_range(
    south_lat=south_lat,
    north_lat=north_lat,
    west_lon=west_lon,
    east_lon=east_lon,
)
print(data.site_name.values)

['Honolulu_Hawaii' 'Hilo_Hawaii' 'Midway' 'Johnston' 'Kahului'
 'Nawiliwili' 'Mokuoloe' 'French_Frigate' 'Kawaihae' 'French_Frigate'
 'Barbers_Point_HI' 'Honolulu_Kewalo' 'Port_Allen' 'Kaumalapau_HI'
 'Honolulu_Hawaii' 'Honolulu_Pier_45' 'Honolulu' 'Hilo' 'Nawiliwili'
 'Sand_Island' 'Kahului' 'Mokuoloe' 'Kawaihae' 'Johnston_Atoll'
 'Port_Allen' 'Kaumalapau_Harbor' 'Kaunakakai_Harbor' 'Laiemaloo'
 'Fort_Kamehameha' 'Ford_Island']
